### Imports

In [4]:
import pandas as pd
from pymongo import MongoClient
import time
from bson.objectid import ObjectId
import datetime
import numpy as np
import sqlite3
from time import time
import pprint

### Connect to the DB

In [5]:
# connection to the database
client = MongoClient()

db_name = "PoliceShootings"

db = client[db_name]

## Simple Queries

#### Simple Query 1 - Unarmed Males under 18 killed by Police

In [6]:
query = { '$and': 
         [ { 'age': { '$lt': 18 } }, {
             'gender': 'M' }, {
             'armed': 'unarmed' }] }
killings_collection = db['PoliceKillings']
start = time()
my_result = killings_collection.find(query).sort("age", 1)
for x in my_result:
    print(f"({x['name']}, {int(x['age'])})")

(Jeremy Mardis, 6)
(Jordan Edwards, 15)
(Jose Raul Cruz, 16)
(Deven Guilford, 17)
(David Joseph, 17)
(Armando Garcia-Muro, 17)


#### Simple Query 2 - Women killed by Police while not fleeing and not armed

In [7]:
query = { '$and': 
         [ { 'flee': 'Not fleeing' }, {
             'gender': 'F' }, {
             'armed': 'unarmed' }] }
killings_collection = db['PoliceKillings']
start = time()
my_result = killings_collection.find(query).sort("age", 1)
for x in my_result:
    print(f"({x['name']}, {int(x['age'])})")

(Ciara Meyer, 12)
(Alteria Woods, 21)
(India Kager, 28)
(Autumn Steele, 34)
(Justine Damond, 40)


## Complex queries


#### Complex Query 1 - People armed with a knife shot by police in the state of Texas

In [8]:
response = db['PoliceKillings'].aggregate([
    
    {
        "$lookup":
        {
            "from": "City",       
            "localField": "city_id",  
            "foreignField": "_id", 
            "as": "c"
        },
    },
      {  
          "$unwind": 
          { "path": '$c', "preserveNullAndEmptyArrays": True } 
      },
    {
        "$lookup":
        {
            "from": "State",      
            "localField": "c.state_id",   
            "foreignField": "_id",
            "as": "s"  
        }
    },
      {  
          "$unwind": 
          { "path": '$s', "preserveNullAndEmptyArrays": True } 
      },
    {
      '$match': 
      { 
        '$and':
        [
          {'s.acronym': 'TX' },
          {'armed': 'knife' }
        ] 
      }
    }
])

for x in response:
    print(f"({x['name']}, {x['age']}, {x['s']['acronym']}, {x['armed']})")   

(Richard McClendon, 43.0, TX, knife)
(Daniel Brumley, 27.0, TX, knife)
(Kristiana Coignard, 17.0, TX, knife)
(Jose Antonio Espinoza Ruiz, 56.0, TX, knife)
(Kamal Dajani, 26.0, TX, knife)
(Randall Waddel, 49.0, TX, knife)
(Randall Lance Hughes, 48.0, TX, knife)
(Roger Albrecht, nan, TX, knife)
(Henry Reyna, 49.0, TX, knife)
(Michael Clyde Lynch, 37.0, TX, knife)
(Tyler Hunkin, 29.0, TX, knife)
(Gregory Mathis, 36.0, TX, knife)
(Jose Angel Vallarta, 30.0, TX, knife)
(Martin Gomez, 46.0, TX, knife)
(Medger Blake, 41.0, TX, knife)
(Morgan London Rankins, 30.0, TX, knife)
(Epthen Lamont Johnson, 40.0, TX, knife)
(Ray Valdez, 55.0, TX, knife)
(Rodney Henderson, 48.0, TX, knife)
(Emmett Edward Hall, 60.0, TX, knife)
